# Brian McCollum, Estefan Gonzales, Cyrus McCormick
## CS429 Project1: Random Forests

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from enum import Enum
from sklearn.model_selection import train_test_split

from collections import defaultdict

import src.cart

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
sample_df = pd.read_csv("dataset/agaricus-lepiota - sample_solution.csv")
test_df = pd.read_csv("dataset/agaricus-lepiota - testing.csv")
train_df = pd.read_csv("dataset/agaricus-lepiota - training.csv")
# split the dataframe into training and validation sets
train_df, validation_df = train_test_split(train_df, test_size=0.2)

### Node class

In [3]:
class AttrNode:
    def __init__(self, name: str, info_gain: float, is_leaf: bool=False) -> None:
        self.name: str = name
        self.info_gain: float = info_gain
        self.children: dict = dict()
        self.is_leaf: bool = is_leaf

    def __str__(self) -> str:
        return f"attribute=[name:{self.name}, info_gain:{self.info_gain}], is_leaf={self.is_leaf}]"

## Information Gain

In [4]:
class iGainType(Enum):
    entropy = 0
    gini = 1
    misclass = 2

### Gini Index
$1 - \sum_{i=1}^{m} p_i^2$

In [5]:
def gini_index(dataset, classifier):
    n = len(dataset.index)
    prob = 0
    class_dict = defaultdict(int)
    for datum in dataset.loc[:, classifier]:
        class_dict[datum] += 1
    for key in class_dict:
        prob += ((class_dict[key] / n) ** 2)
    return 1 - prob

### Entropy
$\sum_{i=1}^{m} - p_i log_2 p_i$

In [6]:
def entropy(dataset, classifier):
    n = len(dataset.index)
    prob = 0
    class_dict = defaultdict(int)
    for datum in dataset.loc[:, classifier]:
        class_dict[datum] += 1
    for key in class_dict:
        p = (class_dict[key] / n)
        prob += (p *  np.log2(p))

    return -1 * prob

### Misclassification error

$1 - \max\limits_k p_k$

In [7]:
def misclassification_error(dataset, classifier):
    n = len(dataset.index)
    probs = []
    class_dict = defaultdict(int)
    for datum in dataset.loc[:, classifier]:
        class_dict[datum] += 1
    for key in class_dict:
        probs.append((class_dict[key] / n))

    return 1 - max(probs)

## Tree creation

#### Defining helper functions for tree creation

In [8]:
def split(dataset, attr, classifier, criteria: iGainType = iGainType.entropy):
    n = len(dataset.index)
    attr_dict = defaultdict(int)
    split_value = 0
    for datum in dataset.loc[:, attr]:
        attr_dict[datum] += 1
    for attr_value in attr_dict:
        # proportion
        weight = attr_dict[attr_value] / n
        # choose all examples with this attr value
        subset = dataset.loc[dataset[attr] == attr_value]

        # impurity
        attr_v = 0
        if criteria == iGainType.gini:
            attr_v = gini_index(subset, classifier)
        elif criteria == iGainType.entropy:
            attr_v = entropy(subset, classifier)
        elif criteria == iGainType.misclass:
            attr_v = misclassification_error(subset, classifier)
            # handle here w weight

        # info gain
        split_value += (weight * attr_v)

    return split_value

def split_values(dataset, classifier, attributes, criteria: iGainType = iGainType.entropy):
    split_vals = dict()
    for col in attributes:
        if col == classifier:
            continue

        split_vals[col] = split(
            dataset, col, classifier, criteria)

    return split_vals

def majority_classification(data_set, classifier):
    classification_count = {}
    for datum in data_set.loc[:, classifier]:
        if datum not in classification_count:
            classification_count[datum] = 1
        else:
            classification_count[datum] += 1
    majority = max(classification_count.values())
    for key in classification_count.keys():
        if classification_count[key] == majority:
            return key
    return -1

def best_attr(attr_split_values):
    argmin = min(attr_split_values.values())
    for k in attr_split_values.keys():
        if attr_split_values[k] == argmin:
            return k
    print('none found')
    return ''

def homogeneous(data_set, classifier):
    class_set = {val for val in data_set.loc[:, classifier]}
    return len(class_set) == 1

def classify(root: AttrNode, example: {}):
    while not root.is_leaf:
        ex_value = example[root.name]
        root = root.children[ex_value]
    return root.name

In [9]:
def create_decision_tree(data_set, classifier, attributes, examples, criteria: iGainType = iGainType.entropy):
    if homogeneous(data_set, classifier) or len(attributes) == 1:
        cls = majority_classification(data_set, classifier)
        return AttrNode(cls, 0, True)

    attr_split_values = split_values(
        data_set,
        classifier,
        attributes,
        criteria
    )
    best_classifier = best_attr(attr_split_values)
    values = {val for val in examples.loc[:, best_classifier]}
    root = AttrNode(best_classifier, attr_split_values[best_classifier])

    for value in values:
        value_subset = data_set.loc[data_set[best_classifier] == value]
        if len(value_subset.index) == 0:
            # no data records in subset, classification node
            # w/ value set to most common class at root node
            maj_class = majority_classification(data_set, classifier)
            child = AttrNode(maj_class, 0, True)
            root.children[value] = child
        else:
            # sub trees for remaining attributes
            root.children[value] = create_decision_tree(value_subset,
                                                        classifier,
                                                        attributes.values().remove(best_classifier),
                                                        examples)
    return root

### Train the model

In [10]:
training_data = train_df.loc[:, train_df.columns != 'id']
gini_root = create_decision_tree(training_data, 'class', training_data.columns, training_data, iGainType.gini)
entropy_root = create_decision_tree(training_data, 'class', training_data.columns, training_data, iGainType.entropy)
misclass_root = create_decision_tree(training_data, 'class', training_data.columns, training_data, iGainType.misclass)

### Predicition

In [11]:
validation_dict = validation_df.to_dict('records')
test_dict = test_df.to_dict('records')

total_correct = 0
total = len(validation_dict)

for prediction in validation_dict:
    actual_class = classify(misclass_root, prediction)
    expected_class = prediction['class']
    if actual_class == expected_class:
        total_correct += 1

In [12]:
print(gini_root)
print(entropy_root)
print(misclass_root)

accuracy = total_correct / total
print(f'\ncorrect={total_correct}, total={total}, accuracy={accuracy}')

attribute=[name:odor, info_gain:0.026498874477435874], is_leaf=False
attribute=[name:odor, info_gain:0.08751921443519706], is_leaf=False
attribute=[name:odor, info_gain:0.013684210526315767], is_leaf=False

correct=1425, total=1425, accuracy=1.0
